## Calibration and rectification of the checkerboard images

In [3]:
import cv2
import numpy as np
import glob
import os

checkerboards_2 = [
    (137, 124, 310, 332, 11, 7),  
    (371, 171, 476, 317, 11, 7),  
    (456, 65, 638, 149, 7, 5),    
    (796, 134, 915, 224, 7, 5),    
    (1032, 69, 1205, 150, 7, 5), 
    (1233, 164, 1317, 447, 5, 15),
    (1327, 187, 1386, 358, 5, 7), 
    (1102, 259, 1189, 372, 5, 7),  
    (998, 385, 1152, 461, 7, 5),   
    (806, 298, 899, 420, 5, 7),    
    (681, 298, 775, 415, 5, 7),    
    (508, 272, 611, 392, 5, 7),   
    (501, 388, 673, 491, 7, 5),    
]

checkerboards_3 = [
    (74, 147, 239, 329, 11, 7),  
    (303, 181, 403, 315, 11, 7), 
    (367, 80, 539, 162, 7, 5),  
    (701, 134, 826, 222, 7, 5), 
    (911, 65, 1072, 142, 7, 5), 
    (1125, 149, 1197, 431, 5, 15), 
    (1206, 173, 1269, 335, 5, 7),
    (1000, 244, 1091, 363, 5, 7),
    (886, 369, 1039, 446, 7, 5), 
    (721, 298, 815, 408, 5, 7), 
    (609, 294, 691, 407, 5, 7), 
    (441, 271, 533, 383, 5, 7), 
    (415, 377, 575, 476, 7, 5), 
]

def preprocess_image(gray_img):
    """Apply unsharp masking for sharper edges"""
    blurred = cv2.GaussianBlur(gray_img, (5, 5), 0)
    
    # Unsharp mask: original + alpha * (original - blurred)
    alpha = 4 # Sharpening strength
    sharpened = cv2.addWeighted(gray_img, 1 + alpha, blurred, -alpha, 0)
    
    diff = cv2.subtract(gray_img, blurred)
    enhanced = cv2.subtract(sharpened, diff)
    
    equalized = cv2.equalizeHist(enhanced)
    return equalized

def detect_single_checkerboard(gray_img, checkerboard_roi, criteria):
    """Detect a single checkerboard in the full image using ROI hint"""
    x1, y1, x2, y2, nv, nh = checkerboard_roi
    
    # Create object points for this checkerboard
    objp = np.zeros((nh * nv, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nv, 0:nh].T.reshape(-1, 2)
    
    # Extract ROI for detection hint (but don't crop the full image)
    roi_gray = gray_img[y1:y2, x1:x2]
    flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
    # Try to find checkerboard corners in the ROI
    ret, corners = cv2.findChessboardCorners(roi_gray, (nv, nh), None, flags)

    if ret:
        # Adjust corners to full image coordinates
        corners[:, 0, 0] += x1
        corners[:, 0, 1] += y1
        # Refine corners on the full image (not cropped)
        corners_refined = cv2.cornerSubPix(gray_img, corners, (5, 5), (-1, -1), criteria)
        img_display = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        # cv2.drawChessboardCorners(img_display, (nv, nh), corners_refined, ret)
        # cv2.imshow("Detected Checkerboard", img_display)
        # cv2.waitKey(1000)
        return objp, corners_refined
    
    return None, None

def gather_calibration_data_cycling(image_dir, checkerboards):
    """Gather detections cycling through one checkerboard per image"""
    images = sorted(glob.glob(os.path.join(image_dir, '*.png')))
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    all_objpoints = []
    all_imgpoints = []
    num_checkerboards = len(checkerboards)
    
    print(f"Processing {len(images)} images with {num_checkerboards} checkerboards...")
    print(f"Cycling through checkerboards deterministically")
    
    for i, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Preprocess image
        processed = preprocess_image(gray)
        
        # Select checkerboard for this image (cycle through them)
        checkerboard_idx = i % num_checkerboards
        checkerboard = checkerboards[checkerboard_idx]
        
        # Detect the selected checkerboard
        objp, imgp = detect_single_checkerboard(processed, checkerboard, criteria)
        
        if objp is not None:
            all_objpoints.append(objp)
            all_imgpoints.append(imgp)
            print(f"  Image {i+1}: Detected checkerboard {checkerboard_idx} (size {checkerboard[4]}x{checkerboard[5]})")
        else:
            print(f"  Image {i+1}: Failed to detect checkerboard {checkerboard_idx}")
    
    print(f"Total successful detections: {len(all_objpoints)}/{len(images)}")
    return all_objpoints, all_imgpoints, gray.shape[::-1]

def calibrate_camera_robust(objpoints, imgpoints, image_size, camera_name):
    """Calibrate camera with robust outlier rejection"""
    print(f"\n{'='*60}")
    print(f"Calibrating {camera_name}")
    print(f"{'='*60}")
    
    # Initial calibration with all data
    print(f"Initial calibration with {len(objpoints)} views...")
    flags = cv2.CALIB_RATIONAL_MODEL  # Better distortion model
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints, imgpoints, image_size, None, None, flags=flags
    )
    print(f"\nFinal {camera_name} Results:")
    print(f"  RMS Error: {ret:.4f}")
    print(f"  Views Used: {len(objpoints)}")
    print(f"  Camera Matrix:\n{mtx}")
    print(f"  Distortion: {dist.ravel()}")
    
    return ret, mtx, dist, objpoints, imgpoints

def gather_stereo_data_cycling(images_2, images_3, checkerboards_2, checkerboards_3):
    """Gather synchronized stereo detections cycling through checkerboards"""
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    objpoints = []
    imgpoints_2 = []
    imgpoints_3 = []
    num_checkerboards = len(checkerboards_2)
    
    print("\nGathering synchronized stereo detections...")
    print(f"Cycling through {num_checkerboards} checkerboard pairs")
    
    for i, (fname_2, fname_3) in enumerate(zip(images_2, images_3)):
        img_2 = cv2.imread(fname_2)
        img_3 = cv2.imread(fname_3)
        gray_2 = cv2.cvtColor(img_2, cv2.COLOR_BGR2GRAY)
        gray_3 = cv2.cvtColor(img_3, cv2.COLOR_BGR2GRAY)
        
        # Preprocess both images
        processed_2 = preprocess_image(gray_2)
        processed_3 = preprocess_image(gray_3)
        
        # Select checkerboard for this image pair (cycle through them)
        checkerboard_idx = i % num_checkerboards
        
        # Detect in both cameras
        objp_2, imgp_2 = detect_single_checkerboard(processed_2, checkerboards_2[checkerboard_idx], criteria)
        objp_3, imgp_3 = detect_single_checkerboard(processed_3, checkerboards_3[checkerboard_idx], criteria)
        
        # Only keep if detected in BOTH cameras
        if objp_2 is not None and objp_3 is not None:
            if objp_2.shape == objp_3.shape:  # Verify same board size
                objpoints.append(objp_2)
                imgpoints_2.append(imgp_2)
                imgpoints_3.append(imgp_3)
                print(f"  Image pair {i+1}: Synchronized checkerboard {checkerboard_idx}")
            else:
                print(f"  Image pair {i+1}: Size mismatch for checkerboard {checkerboard_idx}")
        else:
            print(f"  Image pair {i+1}: Failed to detect checkerboard {checkerboard_idx} in both cameras")
    
    print(f"Total synchronized detections: {len(objpoints)}/{len(images_2)}")
    return objpoints, imgpoints_2, imgpoints_3

# Main calibration pipeline
if __name__ == "__main__":
    # Step 1: Individual camera calibrations
    print("\n" + "="*60)
    print("STEP 1: Individual Camera Calibrations")
    print("="*60)
    
    # Camera 2
    obj_2, img_2, size_2 = gather_calibration_data_cycling(
        '34759_final_project_raw/calib/image_02/data', checkerboards_2
    )
    rms_2, mtx_2, dist_2, obj_2_filtered, img_2_filtered = calibrate_camera_robust(
        obj_2, img_2, size_2, "Camera 2"
    )
    
    # Camera 3
    obj_3, img_3, size_3 = gather_calibration_data_cycling(
        '34759_final_project_raw/calib/image_03/data', checkerboards_3
    )
    rms_3, mtx_3, dist_3, obj_3_filtered, img_3_filtered = calibrate_camera_robust(
        obj_3, img_3, size_3, "Camera 3"
    )
    
    # Step 2: Stereo calibration
    print("\n" + "="*60)
    print("STEP 2: Stereo Calibration")
    print("="*60)
    
    images_2 = sorted(glob.glob('34759_final_project_raw/calib/image_02/data/*.png'))
    images_3 = sorted(glob.glob('34759_final_project_raw/calib/image_03/data/*.png'))
    
    obj_stereo, img_2_stereo, img_3_stereo = gather_stereo_data_cycling(
        images_2, images_3, checkerboards_2, checkerboards_3
    )
    
    print("\nPerforming stereo calibration...")
    flags = cv2.CALIB_FIX_INTRINSIC  # Use individual calibrations
    ret_stereo, mtx_2_s, dist_2_s, mtx_3_s, dist_3_s, R, T, E, F = cv2.stereoCalibrate(
        obj_stereo, img_2_stereo, img_3_stereo,
        mtx_2, dist_2, mtx_3, dist_3, size_2,
        criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6),
        flags=flags
    )
    
    print(f"\nStereo Calibration Results:")
    print(f"  RMS Error: {ret_stereo:.4f}")
    print(f"  Baseline: {np.linalg.norm(T):.4f} units")
    print(f"  Rotation (deg): {np.linalg.norm(cv2.Rodrigues(R)[0]) * 180 / np.pi:.4f}")
    
    # Step 3: Stereo rectification
    print("\n" + "="*60)
    print("STEP 3: Stereo Rectification")
    print("="*60)
    
    H, W = size_2[1], size_2[0]
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
        mtx_2, dist_2, mtx_3, dist_3, (W, H), R, T, alpha=0.25
    )
    
    map1_2, map2_2 = cv2.initUndistortRectifyMap(mtx_2, dist_2, R1, P1, (W, H), cv2.CV_16SC2)
    map1_3, map2_3 = cv2.initUndistortRectifyMap(mtx_3, dist_3, R2, P2, (W, H), cv2.CV_16SC2)
    
    # Create output directories
    os.makedirs('rectified/image_02', exist_ok=True)
    os.makedirs('rectified/image_03', exist_ok=True)
    
    print("Rectifying images...")
    for i, (fname_2, fname_3) in enumerate(zip(images_2, images_3)):
        img_2 = cv2.imread(fname_2)
        img_3 = cv2.imread(fname_3)
        
        rectified_2 = cv2.remap(img_2, map1_2, map2_2, cv2.INTER_LINEAR)
        rectified_3 = cv2.remap(img_3, map1_3, map2_3, cv2.INTER_LINEAR)
        
        cv2.imwrite(f'rectified/image_02/rectified_2_{i:04d}.png', rectified_2)
        cv2.imwrite(f'rectified/image_03/rectified_3_{i:04d}.png', rectified_3)
    
    print("Done! Check the 'rectified' folder for output.")


STEP 1: Individual Camera Calibrations
Processing 19 images with 13 checkerboards...
Cycling through checkerboards deterministically
  Image 1: Detected checkerboard 0 (size 11x7)
  Image 2: Detected checkerboard 1 (size 11x7)
  Image 3: Detected checkerboard 2 (size 7x5)
  Image 4: Detected checkerboard 3 (size 7x5)
  Image 5: Detected checkerboard 4 (size 7x5)
  Image 6: Detected checkerboard 5 (size 5x15)
  Image 7: Failed to detect checkerboard 6
  Image 8: Detected checkerboard 7 (size 5x7)
  Image 9: Detected checkerboard 8 (size 7x5)
  Image 10: Detected checkerboard 9 (size 5x7)
  Image 11: Detected checkerboard 10 (size 5x7)
  Image 12: Detected checkerboard 11 (size 5x7)
  Image 13: Detected checkerboard 12 (size 7x5)
  Image 14: Detected checkerboard 0 (size 11x7)
  Image 15: Detected checkerboard 1 (size 11x7)
  Image 16: Detected checkerboard 2 (size 7x5)
  Image 17: Detected checkerboard 3 (size 7x5)
  Image 18: Detected checkerboard 4 (size 7x5)
  Image 19: Detected che

## Rectification of the sequence images using the provided camera calibration paramters

In [1]:
import cv2
import numpy as np
import glob
import os

# --- 1. Define Camera Calibration Parameters (from provided file input) ---

def parse_calibration_file(filepath):
    """Parse KITTI calibration file and extract camera parameters."""
    calib_data = {}
    
    with open(filepath, 'r') as f:
        for line in f:
            line = line.strip()
            if ':' not in line:
                continue
            
            key, value = line.split(':', 1)
            key = key.strip()
            value = value.strip()
            
            # Convert numeric values to numpy arrays
            if key.startswith(('K_', 'D_', 'R_', 'T_', 'S_', 'R_rect_', 'P_rect_')):
                values = np.array([float(x) for x in value.split()])
                
                # Reshape matrices appropriately
                if key.startswith('K_') or key.startswith('R_rect_'):
                    values = values.reshape(3, 3)
                elif key.startswith('P_rect_'):
                    values = values.reshape(3, 4)
                elif key.startswith('R_'):
                    values = values.reshape(3, 3)
                elif key.startswith('S_'):
                    values = values.astype(int)  # Size should be integers
                
                calib_data[key] = values
            else:
                calib_data[key] = value
    
    return calib_data

def load_camera_params(calib_file, cam_id):
    """Load parameters for a specific camera from calibration file."""
    calib = parse_calibration_file(calib_file)
    
    cam_str = f'{cam_id:02d}'
    
    K = calib[f'K_{cam_str}']
    D = calib[f'D_{cam_str}']
    R_rect = calib[f'R_rect_{cam_str}']
    P_rect = calib[f'P_rect_{cam_str}']
    S_rect = calib[f'S_rect_{cam_str}']
    
    # S_rect is [width, height]
    size = tuple(S_rect)
    
    return K, D, R_rect, P_rect, size

# Load calibration parameters for Camera 02 and Camera 03
calib_file = '34759_final_project_rect/calib_cam_to_cam.txt'
K_02, D_02, R_rect_02, P_rect_02, ORIGINAL_SIZE = load_camera_params(calib_file, 2)
K_03, D_03, R_rect_03, P_rect_03, ORIGINAL_SIZE = load_camera_params(calib_file, 3)

# --- 2. Rectification Map Generation ---

def generate_rectification_maps(K, D, R_rect, P_rect, size):
    """Generates the Undistortion and Rectification Maps using the provided matrices."""
    #Use the 3x3 rotation part of P_rect as the new camera matrix for initUndistortRectifyMap
    # CV_16SC2 is used for optimal memory and remap speed.
    map1, map2 = cv2.initUndistortRectifyMap(
        K, D, R_rect, P_rect[:3, :3], size, cv2.CV_16SC2
    )
    return map1, map2

# Generate maps for both cameras
print("Generating rectification maps for Camera 02 and 03...")
map1_02, map2_02 = generate_rectification_maps(K_02, D_02, R_rect_02, P_rect_02, ORIGINAL_SIZE)
map1_03, map2_03 = generate_rectification_maps(K_03, D_03, R_rect_03, P_rect_03, ORIGINAL_SIZE)
print("Maps successfully generated.")

# --- 3. Image Loading and Rectification ---

def rectify_all_images():
    """Loads all raw images, rectifies them, and saves the result."""
    
    # Input directories 
    image_dir_02 = '34759_final_project_raw/seq_03/image_02/data'
    image_dir_03 = '34759_final_project_raw/seq_03/image_03/data'
    
    # Output directories 
    output_dir_02 = 'rectified/seq03_image_02'
    output_dir_03 = 'rectified/seq03_image_03'
    
    # Get sorted list of images
    images_02 = sorted(glob.glob(os.path.join(image_dir_02, '*.png')))
    images_03 = sorted(glob.glob(os.path.join(image_dir_03, '*.png')))
    
    # Create output directories
    os.makedirs(output_dir_02, exist_ok=True)
    os.makedirs(output_dir_03, exist_ok=True)
    
    # Ensure synchronization
    min_len = min(len(images_02), len(images_03))
    print(f"\nProcessing {min_len} synchronized image pairs...")

    for i in range(min_len):
        fname_2 = images_02[i]
        fname_3 = images_03[i]
        
        # Load images
        img_02 = cv2.imread(fname_2)
        img_03 = cv2.imread(fname_3)
        
        if img_02 is None or img_03 is None:
            print(f"Skipping pair {i:04d}: Could not load one or both images.")
            continue
            
        # Rectify Camera 02 image
        rectified_02 = cv2.remap(img_02, map1_02, map2_02, cv2.INTER_LINEAR)
        
        # Rectify Camera 03 image
        rectified_03 = cv2.remap(img_03, map1_03, map2_03, cv2.INTER_LINEAR)
        
        # Define output filenames
        # Use the original filename suffix or a sequential number
        base_name = os.path.basename(fname_2)
        
        # Save rectified images
        cv2.imwrite(os.path.join(output_dir_02, f'rectified_02_{base_name}'), rectified_02)
        cv2.imwrite(os.path.join(output_dir_03, f'rectified_03_{base_name}'), rectified_03)
        
    print(f"Rectified images saved to '{output_dir_02}' and '{output_dir_03}'.")


if __name__ == "__main__":
    rectify_all_images()

Generating rectification maps for Camera 02 and 03...
Maps successfully generated.

Processing 376 synchronized image pairs...
Rectified images saved to 'rectified/seq03_image_02' and 'rectified/seq03_image_03'.


## Printing of our calibration parameters for documentation and comparison to the provided

In [ ]:
import numpy as np
from datetime import datetime

def print_calibration_results(camera_idx, mtx, dist, R, T, R_rect, P_rect, image_size, corner_dist=0.0995):
    """
    Print calibration parameters in KITTI format
    
    Args:
        camera_idx: Camera index (0, 1, 2, 3, etc.)
        mtx: Camera intrinsic matrix (3x3)
        dist: Distortion coefficients
        R: Rotation matrix (3x3) - from stereo calibration
        T: Translation vector (3,) - from stereo calibration
        R_rect: Rectification rotation matrix (3x3)
        P_rect: Rectified projection matrix (3x4)
        image_size: (width, height) tuple
        corner_dist: Corner distance in meters (default 0.0995 for KITTI)
    """
    # Get current timestamp
    calib_time = datetime.now().strftime("%d-%b-%Y %H:%M:%S")
    
    print(f"calib_time: {calib_time}")
    print(f"corner_dist: {corner_dist:.6e}")
    
    # Original image size
    width, height = image_size
    print(f"S_{camera_idx:02d}: {width:.6e} {height:.6e}")
    
    # Camera intrinsic matrix K (flattened row-major)
    K_flat = mtx.flatten()
    print(f"K_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in K_flat])}")
    
    # Distortion coefficients (k1, k2, p1, p2, k3)
    D_flat = dist.flatten()
    print(f"D_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in D_flat])}")
    
    # Rotation matrix (flattened row-major)
    R_flat = R.flatten()
    print(f"R_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in R_flat])}")
    
    # Translation vector
    T_flat = T.flatten()
    print(f"T_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in T_flat])}")
    
    # Rectified image size
    rect_width, rect_height = image_size  # Usually same as original
    print(f"S_rect_{camera_idx:02d}: {rect_width:.6e} {rect_height:.6e}")
    
    # Rectification rotation matrix (flattened row-major)
    R_rect_flat = R_rect.flatten()
    print(f"R_rect_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in R_rect_flat])}")
    
    # Rectified projection matrix (flattened row-major, 3x4 = 12 elements)
    P_rect_flat = P_rect.flatten()
    print(f"P_rect_{camera_idx:02d}: {' '.join([f'{x:.6e}' for x in P_rect_flat])}")
    
    print()  # Empty line between cameras

# Have to run after calibration and rectification block above as uses those variables:
if __name__ == "__main__":
    
    # Print results:
    print("\n" + "="*60)
    print("CALIBRATION PARAMETERS (KITTI FORMAT)")
    print("="*60 + "\n")
    
    # Camera 2 
    print_calibration_results(
        camera_idx=2,
        mtx=mtx_2,
        dist=dist_2,
        R=R,  # Rotation from stereo calibration
        T=T,  # Translation from stereo calibration
        R_rect=R1,  # Rectification rotation from stereoRectify
        P_rect=P1,  # Projection matrix from stereoRectify
        image_size=(W, H)
    )
    
    # Camera 3 
    print_calibration_results(
        camera_idx=3,
        mtx=mtx_3,
        dist=dist_3,
        R=R,  # Same rotation (or R.T depending on convention)
        T=T,  # Translation from stereo calibration
        R_rect=R2,  # Rectification rotation from stereoRectify
        P_rect=P2,  # Projection matrix from stereoRectify
        image_size=(W, H)
    )
    
    # Optional: Save to file in KITTI format
    with open('our_calib_cam_to_cam.txt', 'w') as f:
        f.write(f"calib_time: {datetime.now().strftime('%d-%b-%Y %H:%M:%S')}\n")
        f.write(f"corner_dist: 9.950000e-02\n")
        
        # Write camera 2
        f.write(f"S_02: {W:.6e} {H:.6e}\n")
        f.write(f"K_02: {' '.join([f'{x:.6e}' for x in mtx_2.flatten()])}\n")
        f.write(f"D_02: {' '.join([f'{x:.6e}' for x in dist_2.flatten()])}\n")
        f.write(f"R_02: {' '.join([f'{x:.6e}' for x in R.flatten()])}\n")
        f.write(f"T_02: {' '.join([f'{x:.6e}' for x in T.flatten()])}\n")
        f.write(f"S_rect_02: {W:.6e} {H:.6e}\n")
        f.write(f"R_rect_02: {' '.join([f'{x:.6e}' for x in R1.flatten()])}\n")
        f.write(f"P_rect_02: {' '.join([f'{x:.6e}' for x in P1.flatten()])}\n")
        
        # Write camera 3
        f.write(f"S_03: {W:.6e} {H:.6e}\n")
        f.write(f"K_03: {' '.join([f'{x:.6e}' for x in mtx_3.flatten()])}\n")
        f.write(f"D_03: {' '.join([f'{x:.6e}' for x in dist_3.flatten()])}\n")
        f.write(f"R_03: {' '.join([f'{x:.6e}' for x in R.flatten()])}\n")
        f.write(f"T_03: {' '.join([f'{x:.6e}' for x in T.flatten()])}\n")
        f.write(f"S_rect_03: {W:.6e} {H:.6e}\n")
        f.write(f"R_rect_03: {' '.join([f'{x:.6e}' for x in R2.flatten()])}\n")
        f.write(f"P_rect_03: {' '.join([f'{x:.6e}' for x in P2.flatten()])}\n")
    
    print("Calibration parameters saved to 'our_calib_cam_to_cam.txt'")


CALIBRATION PARAMETERS (KITTI FORMAT)

calib_time: 24-Nov-2025 23:16:55
corner_dist: 9.950000e-02
S_02: 1.392000e+03 5.120000e+02
K_02: 1.058041e+03 0.000000e+00 6.914275e+02 0.000000e+00 9.726063e+02 2.580496e+02 0.000000e+00 0.000000e+00 1.000000e+00
D_02: -1.071838e-01 -6.290790e-02 -7.642557e-04 5.990191e-04 8.754877e-02 1.239529e-01 8.721847e-02 -1.906451e-02 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00 0.000000e+00
R_02: 9.993501e-01 2.259417e-02 -2.808610e-02 -2.285893e-02 9.996969e-01 -9.141647e-03 2.787104e-02 9.777724e-03 9.995637e-01
T_02: -3.958239e+00 1.354667e-01 5.508133e-01
S_rect_02: 1.392000e+03 5.120000e+02
R_rect_02: 9.861795e-01 -1.284881e-02 -1.651812e-01 1.189654e-02 9.999064e-01 -6.753063e-03 1.652526e-01 4.694648e-03 9.862401e-01
P_rect_02: 8.694485e+02 0.000000e+00 8.949293e+02 0.000000e+00 0.000000e+00 8.694485e+02 2.649775e+02 0.000000e+00 0.000000e+00 0.000000e+00 1.000000e+00 0.000000e+00

calib_time: 24-Nov-2025 23:16:55
corner_dist: 

## Generating videos of our camera calibration & rectification results on the checkerboard images

In [5]:
import cv2
import numpy as np
import os

def create_comparison_frames(num_images=19, output_dir="comparison_frames"):
    """Save individual frames, then combine with ffmpeg"""
    
    os.makedirs(output_dir, exist_ok=True)
    
    raw_left_template = "34759_final_project_raw/calib/image_02/data/{:010d}.png"
    raw_right_template = "34759_final_project_raw/calib/image_03/data/{:010d}.png"
    rect_left_template = "rectified/image_02/rectified_2_{:04d}.png"
    rect_right_template = "rectified/image_03/rectified_3_{:04d}.png"
    
    for i in range(num_images):
        raw_left = cv2.imread(raw_left_template.format(i))
        raw_right = cv2.imread(raw_right_template.format(i))
        rect_left = cv2.imread(rect_left_template.format(i))
        rect_right = cv2.imread(rect_right_template.format(i))
        
        if any(img is None for img in [raw_left, raw_right, rect_left, rect_right]):
            print(f"Warning: Could not load all images for frame {i}")
            continue
        
        cv2.putText(raw_left, f"Left Raw - {i}", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(rect_left, f"Left Rectified - {i}", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(raw_right, f"Right Raw - {i}", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(rect_right, f"Right Rectified - {i}", (10, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        top_row = np.hstack([raw_left, rect_left])
        bottom_row = np.hstack([raw_right, rect_right])
        combined = np.vstack([top_row, bottom_row])
        
        output_path = os.path.join(output_dir, f"frame_{i:04d}.png")
        cv2.imwrite(output_path, combined)
        print(f"Saved {output_path}")
    
    print(f"\nFrames saved to {output_dir}/")
    print("To create video with ffmpeg, run:")
    print(f"ffmpeg -framerate 2 -i {output_dir}/frame_%04d.png -c:v libx264 -pix_fmt yuv420p output.mp4")

create_comparison_frames()

def frames_to_video(frames_dir="comparison_frames", output_name="rectification_comparison.mp4", fps=2):
    """Convert saved frames to video using OpenCV"""
    
    # Get list of frame files
    frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.png')])
    
    if not frame_files:
        print(f"No frames found in {frames_dir}")
        return
    
    # Read first frame to get dimensions
    first_frame = cv2.imread(os.path.join(frames_dir, frame_files[0]))
    h, w = first_frame.shape[:2]
    
    # Try different codecs
    for codec_name, codec_code, extension in [
        ('H.264', 'H264', '.mp4'),
        ('X264', 'X264', '.mp4'),
        ('MJPEG', 'MJPG', '.avi'),
        ('XVID', 'XVID', '.avi')
    ]:
        try:
            output_file = output_name.replace('.mp4', extension)
            fourcc = cv2.VideoWriter_fourcc(*codec_code)
            out = cv2.VideoWriter(output_file, fourcc, fps, (w, h))
            
            if out.isOpened():
                print(f"Using {codec_name} codec...")
                
                for frame_file in frame_files:
                    frame = cv2.imread(os.path.join(frames_dir, frame_file))
                    out.write(frame)
                    print(f"Writing {frame_file}")
                
                out.release()
                print(f"\n✓ Video created: {output_file}")
                return
            else:
                out.release()
        except:
            continue
    
    print("All codecs failed. Please install ffmpeg.")

# Run this after creating the frames
frames_to_video()

Saved comparison_frames/frame_0000.png
Saved comparison_frames/frame_0001.png
Saved comparison_frames/frame_0002.png
Saved comparison_frames/frame_0003.png
Saved comparison_frames/frame_0004.png
Saved comparison_frames/frame_0005.png
Saved comparison_frames/frame_0006.png
Saved comparison_frames/frame_0007.png
Saved comparison_frames/frame_0008.png
Saved comparison_frames/frame_0009.png
Saved comparison_frames/frame_0010.png
Saved comparison_frames/frame_0011.png
Saved comparison_frames/frame_0012.png
Saved comparison_frames/frame_0013.png
Saved comparison_frames/frame_0014.png
Saved comparison_frames/frame_0015.png
Saved comparison_frames/frame_0016.png
Saved comparison_frames/frame_0017.png
Saved comparison_frames/frame_0018.png

Frames saved to comparison_frames/
To create video with ffmpeg, run:
ffmpeg -framerate 2 -i comparison_frames/frame_%04d.png -c:v libx264 -pix_fmt yuv420p output.mp4
Using H.264 codec...
Writing frame_0000.png


OpenCV: FFMPEG: tag 0x34363248/'H264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


Writing frame_0001.png
Writing frame_0002.png
Writing frame_0003.png
Writing frame_0004.png
Writing frame_0005.png
Writing frame_0006.png
Writing frame_0007.png
Writing frame_0008.png
Writing frame_0009.png
Writing frame_0010.png
Writing frame_0011.png
Writing frame_0012.png
Writing frame_0013.png
Writing frame_0014.png
Writing frame_0015.png
Writing frame_0016.png
Writing frame_0017.png
Writing frame_0018.png

✓ Video created: rectification_comparison.mp4


In [10]:
import cv2
import numpy as np

def create_transition_video(image_index=0, output_name="rectification_evolution.mp4", fps=30):
    """
    Create a video showing smooth transition from raw to rectified for one image pair.
    Shows: Raw -> Blend transition -> Rectified -> Difference map
    
    Args:
        image_index: which image pair to use (0 to 18)
        output_name: output video filename
        fps: frames per second
    """
    
    # Path templates
    raw_left_template = "34759_final_project_raw/calib/image_02/data/{:010d}.png"
    raw_right_template = "34759_final_project_raw/calib/image_03/data/{:010d}.png"
    rect_left_template = "rectified/image_02/rectified_2_{:04d}.png"
    rect_right_template = "rectified/image_03/rectified_3_{:04d}.png"
    
    # Load first image to get dimensions
    first_img = cv2.imread(raw_left_template.format(0))
    if first_img is None:
        print(f"Error: Could not load {raw_left_template.format(0)}")
        return
    
    h, w = first_img.shape[:2]
    
    # Video writer - side by side (left and right cameras)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_name, fourcc, fps, (w * 2, h))
    
    if not out.isOpened():
        print("mp4v codec failed, trying avc1...")
        fourcc = cv2.VideoWriter_fourcc(*'avc1')
        out = cv2.VideoWriter(output_name, fourcc, fps, (w * 2, h))
    
    print(f"Creating evolution video for image pair {image_index}...")
    
    # Number of transition frames between stages
    transition_frames = 30  # 1 second at 30fps
    hold_frames = 15  # 0.5 seconds hold on each stage
    
    i = image_index
    print(f"\nProcessing image pair {i}")
    
    # Load all 4 images
    raw_left = cv2.imread(raw_left_template.format(i))
    raw_right = cv2.imread(raw_right_template.format(i))
    rect_left = cv2.imread(rect_left_template.format(i))
    rect_right = cv2.imread(rect_right_template.format(i))
    
    if any(img is None for img in [raw_left, raw_right, rect_left, rect_right]):
        print(f"Error: Could not load all images for frame {i}")
        return
    
    # Convert to float for blending
    raw_left_f = raw_left.astype(float)
    raw_right_f = raw_right.astype(float)
    rect_left_f = rect_left.astype(float)
    rect_right_f = rect_right.astype(float)
    
    # Create difference images (enhanced for visibility)
    diff_left = cv2.absdiff(raw_left, rect_left)
    diff_right = cv2.absdiff(raw_right, rect_right)
    # Enhance difference visibility
    diff_left = cv2.convertScaleAbs(diff_left, alpha=3)
    diff_right = cv2.convertScaleAbs(diff_right, alpha=3)
    
    # Stage 1: Hold on raw images
    for _ in range(hold_frames):
        frame_left = raw_left.copy()
        frame_right = raw_right.copy()
        cv2.putText(frame_left, f"Left Raw - Frame {i}", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame_right, f"Right Raw - Frame {i}", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        combined = np.hstack([frame_left, frame_right])
        out.write(combined)
    
    # Stage 2: Transition from raw to rectified
    for t in range(transition_frames):
        alpha = t / transition_frames
        
        # Blend images
        blend_left = cv2.addWeighted(raw_left, 1-alpha, rect_left, alpha, 0)
        blend_right = cv2.addWeighted(raw_right, 1-alpha, rect_right, alpha, 0)
        
        cv2.putText(blend_left, f"Left Transitioning... {int(alpha*100)}%", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        cv2.putText(blend_right, f"Right Transitioning... {int(alpha*100)}%", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        combined = np.hstack([blend_left, blend_right])
        out.write(combined)
    
    # Stage 3: Hold on rectified images
    for _ in range(hold_frames):
        frame_left = rect_left.copy()
        frame_right = rect_right.copy()
        cv2.putText(frame_left, f"Left Rectified - Frame {i}", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.putText(frame_right, f"Right Rectified - Frame {i}", (10, 40), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        combined = np.hstack([frame_left, frame_right])
        out.write(combined)
    
    print(f"  Added {hold_frames*3 + transition_frames*2} frames")
    
    out.release()
    
    total_frames = hold_frames * 3 + transition_frames * 2
    print(f"\n✓ Video saved as {output_name}")
    print(f"  Total frames: {total_frames}")
    print(f"  FPS: {fps}")
    print(f"  Duration: {total_frames/fps:.1f} seconds")

if __name__ == "__main__":
    # Create the evolution video for image 0 (change this to use a different image)
    create_transition_video(image_index=0, output_name="rectification_evolution.mp4", fps=30)

Creating evolution video for image pair 0...

Processing image pair 0
  Added 105 frames

✓ Video saved as rectification_evolution.mp4
  Total frames: 105
  FPS: 30
  Duration: 3.5 seconds
